In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,GlobalMaxPooling2D,Add,Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.backend import function
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input, Multiply
from tensorflow.keras.applications import InceptionV3,MobileNet,DenseNet201

In [ ]:
# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def attention_module(input,ratio=8):
    avg_pool = GlobalAveragePooling2D()(input)
    max_pool = GlobalMaxPooling2D()(input)

    channel_avg = Dense(units=input.shape[-1] // ratio, activation='relu',kernel_initializer='he_normal')(avg_pool)
    channel_max = Dense(units=input.shape[-1] // ratio, activation='relu',kernel_initializer='he_normal')(max_pool)

    channel_avg = Dense(units=input.shape[-1], activation='sigmoid')(channel_avg)
    channel_max = Dense(units=input.shape[-1], activation='sigmoid')(channel_max)

    channel_attention = Add()([channel_avg, channel_max])
    channel_attention = Multiply()([input, Reshape((1, 1, input.shape[-1]))(channel_attention)])

    return channel_attention


In [ ]:
def create_Global_attention_augmented_mobilenet(input_shape, num_classes,attention=False):
    # Load MobileNet base model without top layer
    base_model = MobileNet(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    # Attention augmented MobileNet architecture
    input_tensor = Input(shape=input_shape)
    x = base_model(input_tensor)

    # Apply attention module
    if(attention):
        x = attention_module(x)

    # Add classification layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

In [ ]:
def create_Global_InceptionV3_model(input_shape, num_classes,attention=False):
    
    # Load Inception-v3 base model without top layer
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    # Attention augmented Inception-v3 architecture
    input_tensor = Input(shape=input_shape)
    x = base_model(input_tensor)

    # Apply attention module
    if(attention):
        x = attention_module(x)

    # Add classification layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, balanced_accuracy_score, average_precision_score, confusion_matrix

def evaluate_classification(y_true, y_pred, average='macro'):
    """
    Evaluate the classification performance and calculate micro-average, balanced accuracy, and average precision.

    Parameters:
        y_true (numpy array or list): True labels.
        y_pred (numpy array or list): Predicted labels.
        average (str, optional): The averaging strategy to use for average precision.
                                 Possible values are 'macro', 'micro', 'weighted', and None.
                                 Default is 'macro'.

    Returns:
        report (str): The classification report as a string.
        balanced_acc (float): The balanced accuracy.
        avg_precision (float): The average precision.
        micro_avg_precision (float): The micro-average precision.
        micro_avg_recall (float): The micro-average recall.
        micro_avg_f1_score (float): The micro-average F1-score.
    """
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=1)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    avg_precision = average_precision_score(y_true, y_pred, average=average)

    # Calculate micro-average precision and recall using confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tp_sum = np.sum(np.diag(cm))
    pred_sum = np.sum(cm, axis=0)
    true_sum = np.sum(cm, axis=1)
    micro_avg_precision = tp_sum / pred_sum.sum()
    micro_avg_recall = tp_sum / true_sum.sum()
    micro_avg_f1_score = 2 * (micro_avg_precision * micro_avg_recall) / (micro_avg_precision + micro_avg_recall)

    return report, balanced_acc, avg_precision, micro_avg_precision, micro_avg_recall, micro_avg_f1_score


In [ ]:
# 1. Load and split the dataset
train_data_dir = '/kaggle/input/eye-diseases-classification/dataset'
#validation_data_dir = 'd:/chaman/cataract/test'
input_shape = (224, 224)
batch_size = 32
num_classes=4


In [ ]:
# 2. Preprocess the images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # 20% validation split
)

#validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical',  # Updated to 'categorical'
    subset="training"
)


validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Subset for validation data
)



In [ ]:
class_counts = train_generator.classes
class_indices = train_generator.class_indices
num_classes = len(class_indices)

# Create a dictionary to store the counts for each class
class_counts_dict = {class_name: np.sum(class_counts == class_idx) for class_name, class_idx in class_indices.items()}

# Print the class counts
for class_name, count in class_counts_dict.items():
    print(f"Class '{class_name}': {count} samples")

# Alternatively, you can simply print the 'class_counts_dict' dictionary
print(class_counts_dict)


In [ ]:

MobileNetModel = create_Global_attention_augmented_mobilenet(input_shape + (3,), num_classes,attention=True)

# 4. Compile the model
MobileNetModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
InceptionV3Model = create_Global_InceptionV3_model(input_shape + (3,), num_classes,attention=True)

InceptionV3Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=40, verbose=0, restore_best_weights=True)

In [ ]:
# 5. Train the model
epochs = 100

# Create a directory to save the best model weights during training
os.makedirs('models', exist_ok=True)
model_checkpoint = ModelCheckpoint('models/best_GlobalmobilnetmodelD1.h5', save_best_only=True, save_weights_only=True)

history = MobileNetModel.fit(
    train_generator,
    #steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    #validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[model_checkpoint,early_stopping]
)


In [ ]:
# Create a directory to save the best model weights during training
os.makedirs('models', exist_ok=True)
model_checkpoint = ModelCheckpoint('models/best_GlobalInceptionModelD1.h5', save_best_only=True, save_weights_only=True)

InceptionV3history = InceptionV3Model.fit(
    train_generator,
    #steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    #validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[model_checkpoint]
)


In [ ]:
MobileNetModel.save('GAAMD1.h5')
InceptionV3Model.save('GAAIV3D1.h5')

In [ ]:
# 6. Evaluate the model
MobileNetEvaluation = MobileNetModel.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(MobileNetEvaluation[1] * 100))

In [ ]:
# 6. Evaluate the model
InceptionV3Evaluation = InceptionV3Model.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(InceptionV3Evaluation[1] * 100))

In [ ]:
def create_Global_attention_augmented_Densenet201_model(input_shape, num_classes,attention=False):
    
    # Load Inception-v3 base model without top layer
    base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    # Attention augmented Inception-v3 architecture
    input_tensor = Input(shape=input_shape)
    x = base_model(input_tensor)

    # Apply attention module
    if(attention):
        x=chanel_attention(x,ratio=16)
    #x = channel_attention(x,16)

    # Add classification layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model


In [ ]:

DenseNetModel = create_Global_attention_augmented_Densenet201_model(input_shape + (3,), num_classes,attention=True)

# 4. Compile the model
DenseNetModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# 5. Train the model
epochs = 100

# Create a directory to save the best model weights during training
os.makedirs('models', exist_ok=True)
model_checkpoint = ModelCheckpoint('models/best_GlobalDenseNetmodelD1.h5', save_best_only=True, save_weights_only=True)

history = DenseNetModel.fit(
    train_generator,
    #steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    #validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[model_checkpoint,early_stopping]
)
